In [1]:
import os
os.environ['PINECONE_API_KEY']='c46f8c74-f32b-4b43-ba41-c6f8808124b3'

In [2]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = 'medical-chatbot'

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name="medical-chatbot",
        dimension=384, 
        metric="dotproduct", 
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        ) 
    ) 


e:\Projects-GenAI\Medical_Chatbot_Llama2\env\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:

from langchain_pinecone import PineconeVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
import os
import time

def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

extracted_data = load_pdf('data')

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

text_chunks = text_split(extracted_data)

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

    return embeddings

embeddings = download_huggingface_embeddings()

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name="medical-chatbot",
    embedding=embeddings, 
    namespace="default" 
)

time.sleep(1)


e:\Projects-GenAI\Medical_Chatbot_Llama2\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [4]:
from pinecone.grpc import PineconeGRPC as Pinecone
import os

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index = pc.Index("medical-chatbot")
namespace = "default"

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[0], 
        namespace=namespace, 
        top_k=1,
        include_values=True,
        include_metadata=True
    )
    print(query)

{'matches': [{'id': '0001c179-adaf-4c4b-bf86-89bdf3247e5d',
              'metadata': {'page': 533.0,
                           'source': 'data\\Medical_book.pdf',
                           'text': 'poor circulation, but it should not be '
                                   'considered a substi-tute for other '
                                   'treatments the physician recommends, '
                                   'suchas physical therapy or surgery.\n'
                                   'This medicine may take several weeks to '
                                   'produce\n'
                                   'noticeable results. Be sure to keep taking '
                                   'it as directed,even if it doesn’t seem to '
                                   'be helping.\n'
                                   'Patients being treated with this medicine '
                                   'should not\n'
                                   'smoke, as smoking may worsen the

In [5]:
from langchain.chains import RetrievalQA
from langchain_pinecone import PineconeVectorStore
from langchain.llms import CTransformers

# Initialize a LangChain object for chatting with the LLM
# without knowledge from Pinecone.
llm=CTransformers(model="model\llama-2-7b.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

# Initialize a LangChain object for retrieving information from Pinecone.
knowledge = PineconeVectorStore.from_existing_index(
    index_name="medical-chatbot",
    namespace="default",
    embedding=embeddings
)

# Initialize a LangChain object for chatting with the LLM
# with knowledge from Pinecone. 
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=knowledge.as_retriever()
)

In [ ]:
user_input='what is acne?'
result=qa({"query": user_input})
print("Response : ", result["result"])

In [ ]:
# from pinecone_text.sparse import BM25Encoder

# bm25_encoder = BM25Encoder().default()


In [ ]:
# bm25_encoder.fit([t.page_content for t in text_chunks])

# bm25_encoder.dump('bm25_values.json')

# bm25_encoder = BM25Encoder().load('bm25_values.json')

In [ ]:
# index = pc.Index("medical-chatbot")

In [ ]:
# from langchain.retrievers.pinecone_hybrid_search import PineconeHybridSearchRetriever
# retrieval = PineconeHybridSearchRetriever(embeddings=embeddings, 
#                                           index=index, 
#                                           sparse_encoder=bm25_encoder)

In [ ]:
# retrieval

In [ ]:
# retrieval.add_texts([t.page_content for t in text_chunks])

In [ ]:
# retrieval.invoke('what is acne')